<b>In my solution, I suppose that the time that first customer comes in is 0.<br>
Then use the generator function to generate the next customer arrival time.

In [1]:
import numpy as np
from collections import deque 
import matplotlib.pyplot as plt

<b>Customers arrive at the rate of 6.0/min+/-2.0 minutes intervals, which means the arrival interval is 4+(0, 2\*2) <br>
<b>It takes 12.0 min+/-3.0 min to checkout a customer, which means the checkout time is 9+(0, 2*3)<br>

In [2]:
def time_generator(time_base, time_flu):
    return time_base+np.round(np.random.uniform(0.0, 2*time_flu), 2)
#yeild can return several times if there is a loop, but return just once
print(time_generator(4, 2))
print(time_generator(9, 3))

5.38
10.42


<div class="alert alert-block alert-info">
<b>There are two lists: a “future event list” in time of event order and a “delayed list” in first come first serve order<br>
&nbsp; &nbsp; &nbsp; &nbsp; <b>Arrival event: <br>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; <b>this event becomes an "arrived event";<br>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; <b>add it to the delayed event list”; <br>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; generate a new arrival event and put on the future event list <br>
&nbsp; &nbsp; &nbsp; &nbsp; <b>Get next event from “future event list”, becomes the current to serve<br>
&nbsp; &nbsp; &nbsp; &nbsp; <b>End of service event: <br>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; <b>If there an arrived type event on the delayed list, find its service time and add service time plus current time  to the next event list<br>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; <b>if not the server becomes idle
</div>
a: arrival time<br>
d: delay<br>
b: begin service time<br>
s: service time<br>
w: wait time &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; w=d+s<br>
c: completes service time &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; c=a+w<br>

In [3]:
class market_queue_problem:
    def __init__(self, a0, c0, d0, s0, w0, x):
        np.random.seed(10)
        self.current_service = []     # current serviced customer list
        self.customer_count = 1       # number of customers
        self.delay_time_count = 0     # the sum of the customers delay time 
        self.arrival = 0              #first customer comes time is 0
        self.future_event_list = []   # future envent list
        self.delay_list = []
        self.temp = deque()           #store all the complete time , so that can calculate the delay time
        self.clock = 0.00    #time clock
        self.i = 0     # element index in the delay list
        self.x = x     # the threshold for a register cash window
        self.delay_list.append([a0, c0, d0, s0, w0])
        self.current_service.append([a0, c0, d0, s0, w0])
        self.last_arrival = self.clock   # last arrival time
        self.window = 1        # the open register window numbers
        self.service_com = 0   #ifthere is any service complete at the clock time
        self.window_idle = 0   #if there is any window is idle
        self.temp.appendleft(c0)
    
    def time_generator(self, time_base, time_flu):
        return time_base+np.round(np.random.uniform(0.0, 2*time_flu), 2)
    
    def get_next_service(self):
        if(len(self.future_event_list) != 0):
            self.current_service.append(self.future_event_list[0])
            self.future_event_list.pop(0)
            if(len(self.delay_list) != 0):
                self.future_event_list.append(self.delay_list[0])
                self.delay_list.pop(0)
                self.i = self.i-1
                print("get_next_service: 1", self.i)
                return 3
            else:
                print("get_next_service: 2", self.i)
                return 4
        else:
            if(len(self.delay_list) != 0):
                print("get_next_service: self.i",self.i)
                self.future_event_list.append(self.delay_list[0])
                self.delay_list.pop(0)
                self.i = self.i-1
                self.current_service.append(self.future_event_list[0])
                self.future_event_list.pop(0)
                print("get_next_service: 3", self.i)
                if(len(self.delay_list) != 0):
                    self.future_event_list.append(self.delay_list[0])
                    self.delay_list.pop(0)
                    self.i = self.i-1
                    print("get_next_service: 4", self.i)
                return 2
            else:
                print("get_next_service: 5", self.i)
                return 1

#     def GetArrival(self): 
#         self.arrival = self.time_generator(4,2)
#         self.a[self.i] = self.clock+self.arrival
#         self.delay_list.append(self.i)
#         self.future_event_list.append(self.i)
     
    #def GetService(self):
        # self.s[self.i] = self.time_generator(9,3)

    # check if there is a service complete
    def check_survice_complete(self):
        # 1) if current there is none service which is going, if no service is going:
             #then check if the future event list has no customers waiting, then check the delay list, if all none, return 1
                                                                            #if the delay list is not empty, return 2
             #else the future event list has customers waiting, then service the one first comes, 
                                         #if delay list is not empty, return 3
                                        #if delay list is empty, return 4
        # 2) if current there is service and the clock shows some service completes, then check future and delay list
        if(len(self.current_service) == 0):
            for i in range(0, self.window):
                print("check_survice_complete: get next service becuase no service now")
                ret = self.get_next_service()
            self.service_com = 1    # delay list send customer to be served
        k = len(self.current_service)
        for i in range(0, k):
            if(self.window_idle == 0):                
                if (self.clock >= self.current_service[i][1]):
    #                 self.service_com = 1
                    print("check_survice_complete: event complete time and self.i", self.current_service[i][1], self.i)
                    self.current_service.pop(i)
                    k = k-1
                    i = i-1
                    print("check_survice_complete: get next service when complete one service")
                    ret = self.get_next_service()
                    self.service_com = 1
                    if(ret == 1):
                        break
                else:
                    self.service_com = 0
            else:
                print("check_survice_complete: should not go inside")
                for j in range(0, self.window_idle):
                    ret = self.get_next_service()
                    self.service_com = 1
    
    #calculate the delay time
    def delay_compute(self):
        print("delay_compute: number of customer in the delay list",self.i)
        if(self.last_arrival<self.temp[0]):
            delay_ = self.temp[0]-self.last_arrival
        else:
            delay_ = 0.0
        self.delay_time_count = self.delay_time_count+delay_
        return delay_

    #check if there is a new arrival:c=a+d+s;w=d+s
    def check_new_arrival(self):
        interval = self.time_generator(4,2)
#         if(self.customer_count > 12):                    #this is used to check if the close register window fun works
#             return False
        if(self.clock >= self.last_arrival+interval):
            self.last_arrival = self.last_arrival+interval
            self.i = self.i+1
            self.customer_count=self.customer_count+1
            print("check_new_arrival: new arrival customer count:", self.customer_count)
            service_time = self.time_generator(9,3)
            delay_time = self.delay_compute()
#             a.append(self.last_arrival)
#             c.append(self.last_arrival+service_time)
#             d.append(delay_time)
#             w.append(service_time+delay_time)
            self.delay_list.append([self.last_arrival, self.last_arrival+service_time+delay_time,
                                    delay_time, service_time, service_time+delay_time])  
            self.temp.appendleft(self.last_arrival+service_time+delay_time)
            return True
        else:
            return False
    
    # open new register cash
    def open_new_window(self):
        if(len(self.future_event_list) == 0):
            self.future_event_list.append(self.delay_list[0])
            self.delay_list.pop(0)
            self.i = self.i-1
        self.current_service.append(self.future_event_list[0])
        self.future_event_list.pop(0)
                    
        self.future_event_list.append(self.delay_list[0])
        self.delay_list.pop(0)
        self.i=self.i-1
        self.window = self.window+1
        print("open_new_window: open new window")
    
    def close_idle_window(self):
        if(self.window-self.window_idle == 0):  # one register window and it is idle
            self.window = 1
        else:
            self.window = self.window-self.window_idle
        print("close_idle_window: close window")
            
    def main_fun(self):
        while(self.clock < 1000):
            self.clock = self.clock+0.01
            self.window_idle = self.window-len(self.current_service)
            ret1 = self.check_new_arrival()
            self.check_survice_complete()
            print("new come:",ret1, "idle:",self.window_idle, "window:",self.window, "time:", self.clock)
            if(ret1 == True and len(self.delay_list) == self.x and self.window < 6):
                if(self.service_com == 0):  # the new arrival comes and no service complete at the same time which needs a new register
                    self.open_new_window()
            if(ret1 == False and self.window_idle>0 and self.window>1):
                self.close_idle_window()
        print(self.customer_count)
        return self.delay_time_count/self.customer_count

In [4]:
# market1 = market_queue_problem(0.0,out_time_generator(9,3),0.0,0.0,0.0,4)

In [5]:
# print(market1.main_fun())

In [6]:
obj = []
ini_c = time_generator(9,3)
ret = []
for i in range(5, 100):             #set x from 5 to 99
    obj.append(market_queue_problem(0.0,ini_c,0.0,0.0,0.0,i))

for i in range(0, len(obj)):         #calcualte the ave wait time result for different x
    ret.append(obj[i].main_fun())

new come: False idle: 0 window: 1 time: 0.01
new come: False idle: 0 window: 1 time: 0.02
new come: False idle: 0 window: 1 time: 0.03
new come: False idle: 0 window: 1 time: 0.04
new come: False idle: 0 window: 1 time: 0.05
new come: False idle: 0 window: 1 time: 0.060000000000000005
new come: False idle: 0 window: 1 time: 0.07
new come: False idle: 0 window: 1 time: 0.08
new come: False idle: 0 window: 1 time: 0.09
new come: False idle: 0 window: 1 time: 0.09999999999999999
new come: False idle: 0 window: 1 time: 0.10999999999999999
new come: False idle: 0 window: 1 time: 0.11999999999999998
new come: False idle: 0 window: 1 time: 0.12999999999999998
new come: False idle: 0 window: 1 time: 0.13999999999999999
new come: False idle: 0 window: 1 time: 0.15
new come: False idle: 0 window: 1 time: 0.16
new come: False idle: 0 window: 1 time: 0.17
new come: False idle: 0 window: 1 time: 0.18000000000000002
new come: False idle: 0 window: 1 time: 0.19000000000000003
new come: False idle: 0 

new come: False idle: 0 window: 1 time: 2.0799999999999996
new come: False idle: 0 window: 1 time: 2.0899999999999994
new come: False idle: 0 window: 1 time: 2.099999999999999
new come: False idle: 0 window: 1 time: 2.109999999999999
new come: False idle: 0 window: 1 time: 2.1199999999999988
new come: False idle: 0 window: 1 time: 2.1299999999999986
new come: False idle: 0 window: 1 time: 2.1399999999999983
new come: False idle: 0 window: 1 time: 2.149999999999998
new come: False idle: 0 window: 1 time: 2.159999999999998
new come: False idle: 0 window: 1 time: 2.1699999999999977
new come: False idle: 0 window: 1 time: 2.1799999999999975
new come: False idle: 0 window: 1 time: 2.1899999999999973
new come: False idle: 0 window: 1 time: 2.199999999999997
new come: False idle: 0 window: 1 time: 2.209999999999997
new come: False idle: 0 window: 1 time: 2.2199999999999966
new come: False idle: 0 window: 1 time: 2.2299999999999964
new come: False idle: 0 window: 1 time: 2.239999999999996
new 

new come: False idle: 0 window: 1 time: 3.699999999999965
new come: False idle: 0 window: 1 time: 3.709999999999965
new come: False idle: 0 window: 1 time: 3.7199999999999647
new come: False idle: 0 window: 1 time: 3.7299999999999645
new come: False idle: 0 window: 1 time: 3.7399999999999642
new come: False idle: 0 window: 1 time: 3.749999999999964
new come: False idle: 0 window: 1 time: 3.759999999999964
new come: False idle: 0 window: 1 time: 3.7699999999999636
new come: False idle: 0 window: 1 time: 3.7799999999999634
new come: False idle: 0 window: 1 time: 3.789999999999963
new come: False idle: 0 window: 1 time: 3.799999999999963
new come: False idle: 0 window: 1 time: 3.8099999999999627
new come: False idle: 0 window: 1 time: 3.8199999999999625
new come: False idle: 0 window: 1 time: 3.8299999999999623
new come: False idle: 0 window: 1 time: 3.839999999999962
new come: False idle: 0 window: 1 time: 3.849999999999962
new come: False idle: 0 window: 1 time: 3.8599999999999617
new c

new come: False idle: 0 window: 1 time: 5.189999999999934
new come: False idle: 0 window: 1 time: 5.199999999999934
new come: False idle: 0 window: 1 time: 5.209999999999933
new come: False idle: 0 window: 1 time: 5.219999999999933
new come: False idle: 0 window: 1 time: 5.229999999999933
new come: False idle: 0 window: 1 time: 5.239999999999933
new come: False idle: 0 window: 1 time: 5.2499999999999325
new come: False idle: 0 window: 1 time: 5.259999999999932
new come: False idle: 0 window: 1 time: 5.269999999999932
new come: False idle: 0 window: 1 time: 5.279999999999932
new come: False idle: 0 window: 1 time: 5.289999999999932
new come: False idle: 0 window: 1 time: 5.299999999999931
new come: False idle: 0 window: 1 time: 5.309999999999931
new come: False idle: 0 window: 1 time: 5.319999999999931
new come: False idle: 0 window: 1 time: 5.329999999999931
new come: False idle: 0 window: 1 time: 5.339999999999931
new come: False idle: 0 window: 1 time: 5.34999999999993
new come: Fals

new come: False idle: 0 window: 1 time: 6.819999999999899
new come: False idle: 0 window: 1 time: 6.829999999999899
new come: False idle: 0 window: 1 time: 6.839999999999899
new come: False idle: 0 window: 1 time: 6.849999999999898
new come: False idle: 0 window: 1 time: 6.859999999999898
new come: False idle: 0 window: 1 time: 6.869999999999898
new come: False idle: 0 window: 1 time: 6.879999999999898
new come: False idle: 0 window: 1 time: 6.8899999999998975
new come: False idle: 0 window: 1 time: 6.899999999999897
new come: False idle: 0 window: 1 time: 6.909999999999897
new come: False idle: 0 window: 1 time: 6.919999999999897
new come: False idle: 0 window: 1 time: 6.929999999999897
new come: False idle: 0 window: 1 time: 6.9399999999998965
new come: False idle: 0 window: 1 time: 6.949999999999896
new come: False idle: 0 window: 1 time: 6.959999999999896
new come: False idle: 0 window: 1 time: 6.969999999999896
new come: False idle: 0 window: 1 time: 6.979999999999896
new come: Fa

...

new come: False idle: 0 window: 6 time: 233.59999999993252
new come: False idle: 0 window: 6 time: 233.6099999999325
new come: False idle: 0 window: 6 time: 233.6199999999325
new come: False idle: 0 window: 6 time: 233.6299999999325
new come: False idle: 0 window: 6 time: 233.63999999993248
new come: False idle: 0 window: 6 time: 233.64999999993248
new come: False idle: 0 window: 6 time: 233.65999999993247
new come: False idle: 0 window: 6 time: 233.66999999993246
new come: False idle: 0 window: 6 time: 233.67999999993245
new come: False idle: 0 window: 6 time: 233.68999999993244
new come: False idle: 0 window: 6 time: 233.69999999993243
new come: False idle: 0 window: 6 time: 233.70999999993242
new come: False idle: 0 window: 6 time: 233.7199999999324
new come: False idle: 0 window: 6 time: 233.7299999999324
new come: False idle: 0 window: 6 time: 233.7399999999324
new come: False idle: 0 window: 6 time: 233.74999999993238
new come: False idle: 0 window: 6 time: 233.75999999993238
new

new come: False idle: 0 window: 6 time: 236.0399999999303
new come: False idle: 0 window: 6 time: 236.0499999999303
new come: False idle: 0 window: 6 time: 236.05999999993028
new come: False idle: 0 window: 6 time: 236.06999999993027
new come: False idle: 0 window: 6 time: 236.07999999993027
new come: False idle: 0 window: 6 time: 236.08999999993026
check_survice_complete: event complete time and self.i 236.09 30
check_survice_complete: get next service when complete one service
get_next_service: 1 29
new come: False idle: 0 window: 6 time: 236.09999999993025
new come: False idle: 0 window: 6 time: 236.10999999993024
new come: False idle: 0 window: 6 time: 236.11999999993023
new come: False idle: 0 window: 6 time: 236.12999999993022
new come: False idle: 0 window: 6 time: 236.1399999999302
new come: False idle: 0 window: 6 time: 236.1499999999302
new come: False idle: 0 window: 6 time: 236.1599999999302
new come: False idle: 0 window: 6 time: 236.16999999993018
new come: False idle: 0 

new come: False idle: 0 window: 6 time: 238.50999999992806
new come: False idle: 0 window: 6 time: 238.51999999992805
new come: False idle: 0 window: 6 time: 238.52999999992804
new come: False idle: 0 window: 6 time: 238.53999999992803
new come: False idle: 0 window: 6 time: 238.54999999992802
new come: False idle: 0 window: 6 time: 238.559999999928
new come: False idle: 0 window: 6 time: 238.569999999928
new come: False idle: 0 window: 6 time: 238.579999999928
new come: False idle: 0 window: 6 time: 238.58999999992798
new come: False idle: 0 window: 6 time: 238.59999999992797
new come: False idle: 0 window: 6 time: 238.60999999992796
new come: False idle: 0 window: 6 time: 238.61999999992796
new come: False idle: 0 window: 6 time: 238.62999999992795
new come: False idle: 0 window: 6 time: 238.63999999992794
new come: False idle: 0 window: 6 time: 238.64999999992793
new come: False idle: 0 window: 6 time: 238.65999999992792
new come: False idle: 0 window: 6 time: 238.6699999999279
new 

new come: False idle: 0 window: 6 time: 241.58999999992525
new come: False idle: 0 window: 6 time: 241.59999999992525
new come: False idle: 0 window: 6 time: 241.60999999992524
new come: False idle: 0 window: 6 time: 241.61999999992523
new come: False idle: 0 window: 6 time: 241.62999999992522
new come: False idle: 0 window: 6 time: 241.6399999999252
new come: False idle: 0 window: 6 time: 241.6499999999252
new come: False idle: 0 window: 6 time: 241.6599999999252
new come: False idle: 0 window: 6 time: 241.66999999992518
new come: False idle: 0 window: 6 time: 241.67999999992517
new come: False idle: 0 window: 6 time: 241.68999999992516
new come: False idle: 0 window: 6 time: 241.69999999992515
new come: False idle: 0 window: 6 time: 241.70999999992515
new come: False idle: 0 window: 6 time: 241.71999999992514
new come: False idle: 0 window: 6 time: 241.72999999992513
new come: False idle: 0 window: 6 time: 241.73999999992512
new come: False idle: 0 window: 6 time: 241.7499999999251
n

KeyboardInterrupt: 

In [ ]:
ret

In [ ]:
plt.plot(ret)
plt.ylabel('ave waiting time')
plt.show()